<a href="https://colab.research.google.com/github/HacKP-CyberDome/reubus-app/blob/leos/WebScrapper/News_Feeds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q newspaper3k

     |████████████████████████████████| 215kB 2.7MB/s 
     |████████████████████████████████| 7.4MB 8.7MB/s 
     |████████████████████████████████| 194kB 38.6MB/s 
     |████████████████████████████████| 51kB 5.8MB/s 


In [2]:
import nltk
import requests
import pandas as pd
from bs4 import BeautifulSoup
from newspaper import Article
from datetime import datetime
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
# sheet_url = 'https://docs.google.com/spreadsheets/d/1WzdRJV92WcRMkipxadrPxv5g4j8S2gHy8lYQ7wQe8bo/edit#gid=113004645'
# csv_export_url = sheet_url.replace('/edit#gid=', '/export?format=csv&gid=')
news = pd.read_csv('/content/drive/My Drive/Datasets/NewsFeeds/news_feed.csv')
recent = news.iloc[0,0]
news.head()

,Date,Place,Context,Keywords
0,2020-08-02 15:03:00,Kochi,Kochi: In connection with the diplomatic bagga...,"['sivasankar', 'likely', 'swapnas', 'gold', 'i..."
1,2020-08-02 10:01:00,Kollam,Kollam: In connection with the Uthra murder ca...,"['murder', 'probe', 'dummy', 'team', 'court', ..."
2,2020-08-02 09:34:00,Thiruvananthapuram,Thiruvananthapuram: Senior accountant of Vanch...,"['treasury', 'rs', 'cr', 'investigation', 'cro..."
3,2020-08-01 18:59:00,Kochi,Kochi: In the controversial diplomatic baggage...,"['sivasankar', 'releasing', 'gold', 'baggage',..."
4,2020-08-01 14:55:00,Kozhikode,Kozhikode: Various Malayali organizations in t...,"['join', 'nurse', 'philip', 'america', 'malaya..."


In [4]:
recent = datetime.strptime(recent,'%Y-%m-%d %H:%M:%S')
recent

datetime.datetime(2020, 8, 2, 15, 3)

In [5]:
links = []
url = 'https://english.mathrubhumi.com/news/crime-beat'
page = requests.get(url).text
soup = BeautifulSoup(page)
headline = soup.find("div",{"class":"row listPg-md7-rw"})
for i in headline.find_all('a'):
  if i.get('href'):
    links.append(i.get('href'))

In [6]:
data = {}
count = 0
for i in links:
  urls = 'https://english.mathrubhumi.com/'+i
  page = requests.get(urls).text
  soup = BeautifulSoup(page)
  headline = soup.find("div",{"class":"common_text_en date_outer"})
  if headline:
    date = headline.get_text().strip()
    date_time = datetime.strptime(date[:-4],'%b %d, %Y, %I:%M %p')
    parag = soup.find("div",{"class":"articleBody common_text"})
    para = parag.find_all('p')
    place = para[0].get_text().split(':')[0]
    if date_time>recent:
      article = Article(urls, 'en')
      article.download()
      article.parse()
      article.nlp()
      summary = article.summary
      data[count] = [date_time,place,summary,article.keywords]
      count += 1

In [7]:
links = []
url = 'https://www.onmanorama.com/districts/'
districts = ['alappuzha','ernakulam','idukki','kannur','kasaragod','kollam',
             'kottayam','kozhikode','malappuram','palakkad','pathanamthitta',
             'thiruvananthapuram','thrissur','wayanad']
for d in districts:
  urls = url+d+'.html'
  page = requests.get(urls).text
  soup = BeautifulSoup(page)
  headline = soup.find("div",{"class":"articlelisting section"})
  for i in headline.find_all('a'):
    site = i.get('href')
    if site and site not in links:
      date = site.replace(url+d+'/',"")
      date = date[:10]
      date = datetime.strptime(date,'%Y/%m/%d')
      if date > recent:
        links.append(site)
for i in links:
  urls = i
  page = requests.get(urls).text
  soup = BeautifulSoup(page)
  headline = soup.find("time",{"class":"story-author-date"})
  date_time_str = headline.get_text()
  date_time = datetime.strptime(date_time_str[:-4],'%B %d, %Y %I:%M %p')
  parag = soup.find("div",{"class":"article rte-article"})
  para = parag.find_all('p')
  place = para[0].get_text().split(':')[0]
  article = Article(urls)
  article.download()
  article.parse()
  article.nlp()
  summary = article.summary
  data[count] = [date_time,place,summary,article.keywords]
  count += 1

In [8]:
data

{0: [datetime.datetime(2020, 8, 2, 18, 35),
  'Kochi',
  'Kochi: The National Investigation Agency, on Sunday, arrested two more persons in the gold smuggling case.\nThe NIA team arrested Muhammadali Ibrahim and Muhammadali from Muvattupuzha.\nTheir arrest has confirmed the terrorist links in the smuggling case.\nThe NIA team evaluates the arrest of hand-chopping case accused in gold smuggling case, as a crucial turning pointing in the case.\nThis is considered as a crucial finding in the gold smuggling case.',
  ['terror',
   'evidence',
   'various',
   'arrested',
   'muhammadali',
   'uncovers',
   'smuggling',
   'duo',
   'team',
   'case',
   'ibrahim',
   'prove',
   'nia',
   'persons',
   'gold',
   'links']]}

In [9]:
df = pd.DataFrame.from_dict(data, orient='index',
                       columns=['Date', 'Place', 'Context', 'Keywords'])

In [10]:
df = df.append(news)

In [11]:
for row in df.values:
  row[0] = pd.to_datetime(row[0])

In [12]:
df.sort_values(by='Date',inplace=True,ascending=False)

In [13]:
df.to_csv('/content/drive/My Drive/Datasets/NewsFeeds/news_feed.csv',header=True,index=False,date_format='%Y-%m-%d %H:%M:%S')

In [14]:
df.head(20)

,Date,Place,Context,Keywords
0,2020-08-02 18:35:00,Kochi,"Kochi: The National Investigation Agency, on S...","[terror, evidence, various, arrested, muhammad..."
0,2020-08-02 15:03:00,Kochi,Kochi: In connection with the diplomatic bagga...,"['sivasankar', 'likely', 'swapnas', 'gold', 'i..."
1,2020-08-02 10:01:00,Kollam,Kollam: In connection with the Uthra murder ca...,"['murder', 'probe', 'dummy', 'team', 'court', ..."
2,2020-08-02 09:34:00,Thiruvananthapuram,Thiruvananthapuram: Senior accountant of Vanch...,"['treasury', 'rs', 'cr', 'investigation', 'cro..."
3,2020-08-01 18:59:00,Kochi,Kochi: In the controversial diplomatic baggage...,"['sivasankar', 'releasing', 'gold', 'baggage',..."
4,2020-08-01 14:55:00,Kozhikode,Kozhikode: Various Malayali organizations in t...,"['join', 'nurse', 'philip', 'america', 'malaya..."
5,2020-08-01 14:54:00,Thiruvananthapuram,Sub-treasury officer also lodged a complaint d...,"['swindled', 'subtreasury', 'vanchiyoor', 'fra..."
6,2020-08-01 12:03:00,Tirur,"Tirur: Police arrested a thief, who is an expe...","['vegetable', 'selling', 'kalpakanchery', 'ras..."
7,2020-08-01 09:14:00,Wayanad,The hilly Wayanad district has a total of 310 ...,"['positive', 'total', 'covid19', 'panchayat', ..."
8,2020-08-01 09:05:00,Monippally,"Monippally: Parents of the Keralite nurse, who...","['nurse', 'murdered', 'mathew', 'file', 'merin..."


In [ ]:
class HTMLTableParser:

  def parse_url(self, url):
      response = requests.get(url)
      soup = BeautifulSoup(response.text, 'lxml')
      return [('dfsds',self.parse_html_table(table))\
              for table in soup.find_all('table')]  

  def parse_html_table(self, table):
      n_columns = 0
      n_rows=0
      column_names = []

      # Find number of rows and columns
      # we also find the column titles if we can
      for row in table.find_all('tr'):
          
          # Determine the number of rows in the table
          td_tags = row.find_all('td')
          if len(td_tags) > 0:
              n_rows+=1
              if n_columns == 0:
                  # Set the number of columns for our table
                  n_columns = len(td_tags)
                  
          # Handle column names if we find them
          th_tags = row.find_all('th') 
          if len(th_tags) > 0 and len(column_names) == 0:
              for th in th_tags:
                  column_names.append(th.get_text())

      # Safeguard on Column Titles
      if len(column_names) > 0 and len(column_names) != n_columns:
          raise Exception("Column titles do not match the number of columns")

      columns = column_names if len(column_names) > 0 else range(0,n_columns)
      df = pd.DataFrame(columns = columns,
                        index= range(0,n_rows))
      row_marker = 0
      for row in table.find_all('tr'):
          column_marker = 0
          columns = row.find_all('td')
          for column in columns:
              df.iat[row_marker,column_marker] = column.get_text()
              column_marker += 1
          if len(columns) > 0:
              row_marker += 1
              
      # Convert to float if possible
      for col in df:
          try:
              df[col] = df[col].astype(float)
          except ValueError:
              pass
      
      return df

In [ ]:
place_URLs = [
              'A/','B/','C/','D/','E/','F/','G/','H/','I/','J/','K/a/b/','K/a/c/','K/a/d/','K/a/i/','K/a/k/','K/a/l/','K/a/m/','K/a/n/','K/a/p/',
              'K/a/r/','K/a/s/','K/a/t/','K/a/v/','K/a/y/','K/a/z/','K/e/','K/i/','K/o/','K/u/','L/','M/a/','M/e/','M/i/','M/o/','M/u/','N/','O/',
              'P/a/','P/e/','P/i/','P/o/','P/u/','Q/','R/','S/','T/','U/','V/','W/','Y/']

In [ ]:
column_names = table.columns.to_list()
loc = pd.DataFrame(columns = column_names)

In [ ]:
urls = 'http://www.fallingrain.com/world/IN/13/a/'
for i in place_URLs:
  url = urls + i
  hp = HTMLTableParser()
  table = hp.parse_url(url)[0][1] # Grabbing the table from the tuple
  loc = loc.append(table)

In [ ]:
loc.to_csv('/content/drive/My Drive/Datasets/NewsFeeds/locations.csv',header=True,index=False)

In [ ]:
# article = Article('https://www.onmanorama.com/districts/pathanamthitta/2020/06/10/tiger-dies-pathanamthitta-forest-kill-rubber-contractor.html')
# article.download()
# article.parse()
# print(article.authors)
# print(article.publish_date)
# print(article.text)
# article.nlp()
# print(article.keywords)
# print(article.summary)
# print(article.title)

In [ ]:
# url = 'https://www.onmanorama.com/districts/alappuzha/2020/06/16/covid-fight-kerala-nurse-alappuzha.html'
# page = requests.get(url).text
# soup = BeautifulSoup(page)
# headline = soup.find("time",{"class":"story-author-date"})
# date_time = datetime.strptime(headline.get_text()[:-4],'%B %d, %Y %I:%M %p')
# date_time

In [ ]:
# res = pd.to_datetime(news.iloc[0,0])
# nws = pd.read_csv('/content/drive/My Drive/Datasets/NewsFeeds/news_feed.csv')
# nws.head()
# for row in nws.values:
#   row[0] = pd.to_datetime(row[0])
# type(nws.iloc[0,0])
# nws.to_csv('/content/drive/My Drive/Datasets/NewsFeeds/news_feed.csv',header=True,index=False,date_format='%Y-%m-%d %H:%M:%S')
# ne = pd.read_csv('/content/drive/My Drive/Datasets/NewsFeeds/news_feed.csv',parse_dates=['Date'])
# ne.head()

In [ ]:
# links = [
#          'https://english.mathrubhumi.com/news/crime-beat/pocso-case-accused-jumps-into-sea-during-evidence-collection-1.4922728',
#          'https://english.mathrubhumi.com/news/crime-beat/gold-smuggling-nia-team-takes-sarith-for-evidence-collection-in-tvm-1.4920332',
#          'https://english.mathrubhumi.com/news/crime-beat/dispute-over-funds-for-treatment-police-interrogate-3-including-sajan-kechery-1.4920310',
#          'https://english.mathrubhumi.com/news/crime-beat/neeleswaram-girl-sexually-abused-for-2-yrs-stepmother-also-involved-1.4918395',
#          'https://english.mathrubhumi.com/news/crime-beat/mystery-looms-over-appointment-of-gunman-jayaghosh-1.4918242',
#          'https://english.mathrubhumi.com/news/crime-beat/gold-smuggling-case-malayalam-film-industry-a-beneficiary-reveals-nia-1.4918146',
#          'https://english.mathrubhumi.com/news/crime-beat/dispute-over-fund-for-treatment-source-of-transferred-money-to-be-probed-1.4918123',
#          'https://english.mathrubhumi.com/news/crime-beat/swapna-suresh-allegedly-destroyed-evidence-before-shifting-to-new-house-1.4916142',
#          'https://english.mathrubhumi.com/news/crime-beat/nia-takes-swapna-sandeep-for-evidence-collection-in-tvm-1.4913844',
#          'https://english.mathrubhumi.com/news/crime-beat/plotted-gold-smuggling-with-film-stars-contacted-dharmajan-shamna-kasim-says-accused-1.4913807',
#          'https://english.mathrubhumi.com/news/crime-beat/gold-smuggling-customs-to-interrogate-m-sivasankar-again--1.4913742',
#          'https://english.mathrubhumi.com/news/crime-beat/auto-driver-arrested-for-raping-65-yr-old-woman-in-mukkom-1.4911459',
#          'https://english.mathrubhumi.com/news/crime-beat/swapna-visited-sarith-s-residence-before-moving-to-bengaluru-says-lawyer-1.4911330',
#          'https://english.mathrubhumi.com/news/crime-beat/tobacco-products-found-hidden-inside-cake-at-quarantine-centre-1.4911268',
#          'https://english.mathrubhumi.com/news/crime-beat/gold-smuggling-case-faisal-fareed-s-passport-impounded-uae-issues-travel-ban-1.4909325',
#          'https://english.mathrubhumi.com/news/crime-beat/man-on-bail-who-raped-murdered-woman-in-palakkad-gets-life-sentence-1.4909134',
#          'https://english.mathrubhumi.com/news/crime-beat/online-fraudsters-focus-on-sellers-of-second-hand-items-ask-for-mpin-to-rob-money-1.4908912',
#          'https://english.mathrubhumi.com/news/crime-beat/gold-smuggling-case-sivasankar-directed-to-book-flat-reveals-whatsapp-message-1.4907009',
#          'https://english.mathrubhumi.com/news/crime-beat/more-passengers-caught-smuggling-gold-at-karipur-tvm-airports-1.4906932',
#          'https://english.mathrubhumi.com/news/crime-beat/swapna-phoned-kt-jaleel-call-list-proves-links-between-sivasankar-and-sarith-1.4904698',
#          'https://english.mathrubhumi.com/news/crime-beat/associate-director-booked-for-raping-woman-on-promise-of-marriage-1.4904587',
#          'https://english.mathrubhumi.com/news/crime-beat/-i-did-it-suraj-confesses-to-murdering-uthra-in-front-of-media-1.4904428',
#          'https://english.mathrubhumi.com/news/crime-beat/student-under-quarantine-after-return-from-russia-commits-suicide-in-kottayam-1.4904327',
#          'https://english.mathrubhumi.com/news/crime-beat/-sultan-on-sharechat-arrested-for-abducting-raping-19-year-old-social-media-friend-1.4902277',
#          'https://english.mathrubhumi.com/news/crime-beat/night-party-in-resort-ukraine-dancer-says-she-was-misled-to-perform-belly-dance-1.4902211',
#          'https://english.mathrubhumi.com/news/crime-beat/rameez-acts-as-gold-smugglers-agent-nia-to-probe-gun-trafficking-case-involving-him-1.4902148',
#          'https://english.mathrubhumi.com/news/crime-beat/two-youth-stabbed-in-front-of-father-after-verbal-spat-one-dies-1.4902139',
#          'https://english.mathrubhumi.com/news/crime-beat/huge-gold-haul-again-at-thiruvananthapuram-kannur-airports-1.4902008',
#          'https://english.mathrubhumi.com/news/crime-beat/gold-smuggling-case-alleged-middleman-in-custody-1.4899985',
#          'https://english.mathrubhumi.com/news/crime-beat/nia-vehicle-sustains-puncture-swapna-suresh-shifted-to-another-vehicle-1.4899862',
#          'https://english.mathrubhumi.com/news/crime-beat/gold-smuggling-swapna-suresh-sandeep-nair-brought-to-kerala-through-walayar-1.4899853',
#          'https://english.mathrubhumi.com/news/crime-beat/gold-smuggling-arrest-of-rameez-leads-to-more-clues-swapna-fled-via-perinthalmanna--1.4899830',
#          'https://english.mathrubhumi.com/news/crime-beat/nia-arrests-swapna-sandeep-amid-plans-to-flee-to-nagaland-1.4899790',
#          'https://english.mathrubhumi.com/news/crime-beat/gold-smuggling-case-nia-team-set-out-to-kochi-with-swapna-sandeep-1.4899759',
#          'https://english.mathrubhumi.com/news/crime-beat/gold-smuggling-case-swapna-suresh-sandeep-nair-arrested-from-bengaluru-1.4898205',
#          'https://english.mathrubhumi.com/news/crime-beat/gold-smuggling-case-nia-tightens-noose-search-for-faisal-fareed-in-dubai-1.4897665',
#          'https://english.mathrubhumi.com/news/crime-beat/murder-over-ganja-sale-in-kozhikode-accused-arrested-after-two-and-a-half-years-1.4897596',
#          'https://english.mathrubhumi.com/news/crime-beat/thazhathangadi-robbery-and-murder-man-succumbs-on-40th-day-after-wife-s-death-1.4897455',
#          'https://english.mathrubhumi.com/news/crime-beat/swapna-suresh-had-15-bodyguards-reveals-youth-attacked-by-her-1.4897399',
#          'https://english.mathrubhumi.com/news/crime-beat/dysp-radhakrishna-pillai-to-lead-nia-probe-into-gold-smuggling-1.4897391',
#          'https://english.mathrubhumi.com/news/crime-beat/gold-smuggling-case-conspiracy-hatched-at-flat-near-secretariat-1.4897371',
#          'https://english.mathrubhumi.com/news/crime-beat/gold-smuggling-nia-to-probe-is-links-1.4897252',
#          'https://english.mathrubhumi.com/news/crime-beat/gold-smuggling-nia-to-invoke-uapa-against-accused-1.4894785',
#          'https://english.mathrubhumi.com/news/crime-beat/major-irregularities-in-rs-5-cr-loan-granted-by-adimali-cooperative-bank-come-to-light-1.4894754',
#          'https://english.mathrubhumi.com/news/crime-beat/copy-of-swapna-s-email-seeking-re-export-of-gold-comes-out-1.4892703',
#          'https://english.mathrubhumi.com/news/crime-beat/gold-smuggling-break-the-chain-and-other-modus-operandi-1.4892674',
#          'https://english.mathrubhumi.com/news/crime-beat/gold-smuggling-did-sandeep-swapna-suresh-flee-together--1.4892611',
#          'https://english.mathrubhumi.com/news/crime-beat/didn-t-know-swapna-was-a-gold-smuggler-says-sandeep-s-wife-soumya-1.4892597',
#          'https://english.mathrubhumi.com/news/crime-beat/paravur-ksrtc-driver-found-dead-in-depot-1.4892485',
#          'https://english.mathrubhumi.com/news/crime-beat/man-stabbed-to-death-in-front-of-family-in-mundakayam-1.4892479',
#          'https://english.mathrubhumi.com/news/crime-beat/shamna-blackmail-case-gold-smuggling-was-just-an-offer-accused-had-no-direct-links-1.4890009',
#          'https://english.mathrubhumi.com/news/crime-beat/customs-declines-police-help-to-find-swapna-suresh-seized-gold-kept-in-locker-1.4889981',
#          'https://english.mathrubhumi.com/news/crime-beat/kozhikode-man-loses-rs-14-400-after-recharging-mobile-via-app-1.4889927',
#          'https://english.mathrubhumi.com/news/crime-beat/newlywed-couple-found-dead-at-rented-house-in-alappuzha-1.4889884',
#          'https://english.mathrubhumi.com/news/crime-beat/nun-rape-case-hc-rejects-franco-mulakkal-s-discharge-plea-1.4887722',
#          'https://english.mathrubhumi.com/news/crime-beat/19-year-old-held-for-bid-to-molest-woman-at-quarantine-centre-in-pathanamthitta-1.4887505',
#          'https://english.mathrubhumi.com/news/crime-beat/customs-raid-swapna-suresh-s-flat-again-to-check-visitors-list-1.4887495',
#          'https://english.mathrubhumi.com/news/crime-beat/night-party-belly-dance-6-arrested-released-on-bail-resort-license-to-be-cancelled-1.4887457',
#          'https://english.mathrubhumi.com/news/crime-beat/gold-smuggling-case-sarith-s-phone-trapped-swapna-suresh-cyber-evidences-more-crucial-1.4887382',
#          'https://english.mathrubhumi.com/news/crime-beat/uae-consulate-former-female-staff-is-brain-behind-diplomatic-baggage-gold-smuggling-case-1.4884893',
#          'https://english.mathrubhumi.com/news/crime-beat/woman-lover-arrested-for-abandoning-their-newborn-in-front-of-church-in-adoor-1.4884849',
#          'https://english.mathrubhumi.com/news/crime-beat/gold-seized-from-diplomatic-baggage-in-kerala-for-first-time-37-kg-gold-seized-in-delhi-in-2013-1.4884774',
#          'https://english.mathrubhumi.com/news/crime-beat/50-year-old-murdered-in-thrissur-after-scuffle-over-rs-50-3-arrested-1.4884756',
#          'https://english.mathrubhumi.com/news/crime-beat/20-year-old-held-for-raping-4-year-old-girl-in-mundakayam-1.4882913',
#          'https://english.mathrubhumi.com/news/crime-beat/counterfeit-currency-of-rs-10-lakh-face-value-seized-in-malappuram-3-held-1.4882766',
#          'https://english.mathrubhumi.com/news/crime-beat/rape-case-against-charitable-trust-proprietor-s-husband-in-kottayam-1.4882751',
#          'https://english.mathrubhumi.com/news/crime-beat/after-night-party-row-mystery-looms-over-thannikkod-group-s-quarry-in-chathurangappara-1.4882749',
#          'https://english.mathrubhumi.com/news/crime-beat/man-hacked-to-death-at-rubber-plantation-in-kannur-1.4882735',
#          'https://english.mathrubhumi.com/news/crime-beat/uthra-murder-case-snake-catcher-suresh-submits-application-to-turn-approver-1.4880530',
#          'https://english.mathrubhumi.com/news/crime-beat/women-involved-in-shamna-kasim-blackmail-case-identified-bailed-accused-arrested-again-1.4880519',
#          'https://english.mathrubhumi.com/news/crime-beat/man-throws-his-6-month-old-baby-after-being-asked-to-buy-baby-food-1.4880485',
#          'https://english.mathrubhumi.com/news/crime-beat/false-rape-case-4-held-for-exorting-rs-1-37-lakh-in-adimali-1.4880483',
#          'https://english.mathrubhumi.com/news/crime-beat/300-attend-night-party-belly-dance-violating-covid-19-protocol-in-idukki-businessman-booked-1.4880455',
#          'https://english.mathrubhumi.com/news/crime-beat/3-get-bail-in-shamna-kasim-blackmail-case-producer-who-impersonated-quizzed-1.4880417',
#          'https://english.mathrubhumi.com/news/crime-beat/more-women-involved-in-shamna-kasim-blackmail-case-reveals-ig-vijay-sakhare-1.4878267',
#          'https://english.mathrubhumi.com/news/crime-beat/shamna-kasim-blackmail-case-more-arrests-likely-today-producer-to-be-grilled-1.4878057',
#          'https://english.mathrubhumi.com/news/crime-beat/uthra-murder-case-suraj-s-mother-sister-questioned-again-1.4875821',
#          'https://english.mathrubhumi.com/news/crime-beat/man-drapes-sari-pours-kerosene-and-attempts-to-self-immolate-escapes-when-locals-intervene-1.4875817',
#          'https://english.mathrubhumi.com/news/crime-beat/3-women-prisoners-chased-down-by-police-after-escape-bid-in-ernakulam-1.4875802',
#          'https://english.mathrubhumi.com/news/crime-beat/liquor-smuggled-into-covid-19-quarantine-centre-in-pathanamthitta-1.4875778',
#          'https://english.mathrubhumi.com/news/crime-beat/rafeeq-was-always-on-phone-call-with-shamna-demanded-divorce-from-me-wife-1.4875621',
#          'https://english.mathrubhumi.com/news/crime-beat/shamna-kasim-blackmail-case-police-searching-for-woman-who-posed-as-mother-of-groom--1.4873575',
#          'https://english.mathrubhumi.com/news/crime-beat/abhimanyu-murder-accused-committed-financial-fraud-while-in-hiding-1.4873538',
#          'https://english.mathrubhumi.com/news/crime-beat/man-escapes-after-jumping-into-well-finds-wife-hanging-inside-house-1.4873391'
# ]

In [ ]:
news = pd.read_csv('/content/drive/My Drive/Datasets/NewsFeeds/news_feed.csv')
recent = news.iloc[0,0]
news.head()

In [ ]:
news.head(30)

In [ ]:
for row in news.values:
  print(row[3])